<h4> Save for Storyboard </h4>

In [24]:
import struct, socket
import numpy as np 
import csv, json 
import os 
import urllib2 
import datetime
import operator

import ipywidgets as widgets
# from IPython.html import widgets
# from IPython.html.widgets import interact, interactive, fixed
from IPython.display import display, HTML, clear_output, Javascript 
 
path = os.getcwd().split("/")
t_date = path[len(path)-1] 
spath = os.getcwd()  
spath =  os.environ['LUSER'] + '/ipython/dns/user/'+ t_date + '/'
sconnect = spath + 'dns_scores.csv' 
threat_f = spath + "threats.csv"
anchor = ''
anchor_type = ''

**Interface**

In [25]:
##Expanded search
susp_box   =  widgets.HBox(width=500, height=150)    
susp_h_box =  widgets.Box(width=500, height=200)    
susp_title =  widgets.HTML(value='<h4>Expanded search</h4>')
susp_select = widgets.Select(width=300)

##Search Results
result_title = widgets.HTML(value='<h4>Search results</h4>')     
result_html = widgets.HTML(width=500)
result_box  = widgets.Box(width=500)
result_box.children = [result_title, result_html]               

#Threat Summary
tc_box_main = widgets.Box(width=500, height=200)    
threat_container = widgets.HBox(width=500, height=150)
tc_box_separator = widgets.Box(width=500, height=15)     

yy = t_date[0:4]
mm = t_date[4:6] 
dd = t_date[6:8]

ip_sev={}
dns_sev={}
def start_investigation():
    ips_query = {}  
    display(Javascript("$('.widget-area > .widget-subarea > *').remove();"))
    
    clear_output() 
    with open(sconnect, 'r') as f:
        reader = csv.reader(f, delimiter=',')
        reader.next()
        for row in reader:
            # frame_time, frame_len, ip_dst, dns_qry_name, dns_qry_class,  dns_qry_type, dns_qry_rcode, domain, subdomain, 
            #      0    ,      1   ,    2  ,       3     ,          4   ,         5    ,        6     ,   7   ,    8     ,
            #subdomain_length, query_length, num_periods, subdomain_entropy, top_domain, word, score, query_rep, 
            #      9          ,      10    ,     11     ,    12            ,      13   ,  14 ,   15  ,   16    ,   
            # hh, ip_sev, dns_sev, dns_qry_class_name, dns_qry_type_name, dns_qry_rcode_name, network_context
            # 17,   18  ,   19  ,      20            ,         21       ,           22      ,       23                
            if row[2] not in ips_query and row[18] == '1': 
                    ips_query[row[2]]='i'
            if row[3] not in ips_query and row[19] == '1':
                    ips_query[row[3]]='q' 
            
            if row[2] not in ip_sev: 
                ip_sev[row[2]] = row[15]
            if row[3] not in dns_sev: 
                dns_sev[row[3]] =row[15]
                    
    if len(ips_query) == 0:
        result_html = widgets.HTML(value="There are not high risk results.", width=500)
        result_box  = widgets.Box(width=500, height=150)
        result_box.children = [result_html]        
        display(result_box)              
    else:  
        sorted_dict = sorted(ips_query.items(), key=operator.itemgetter(0))      
        display_controls(sorted_dict)   

        
def display_controls(ip_list): 
    
    susp_title =  widgets.HTML(value='<h4>Expanded search</h4>')
    search_btn = widgets.Button(description='Search')
    susp_box.children = [susp_select,search_btn]
    susp_h_box.children = [susp_title,susp_box] 

    susp_select.options = ip_list
    susp_select.height=min(len(ip_list)*18,150)
    susp_select.selected_label = ip_list[0][0]
    
    display(susp_h_box)
  
    def search_ip(b):  
        global anchor 
        global anchor_type  
        anchor = ''
        anchor_type = ''
        anchor = susp_select.selected_label  
        anchor_type = susp_select.value 
        removeWidget(2)
        removeWidget(1) 
        clear_output()
        global ir_f
        ir_f = spath + "ir-" + anchor.replace(".","_") + ".tsv"
            
        table = "<table border=1><th>IP</th><th>QUERY</th><th>TOTAL</th>"
        
        if anchor_type == 'i':
            hivestr = ("\"set mapred.max.split.size=1073741824;set hive.exec.reducers.max=10;set hive.cli.print.header=true;" +
                   " SELECT COUNT(dns_qry_name) as total, dns_qry_name, ip_dst, 0 as sev FROM duxbury.dns " +
                   " WHERE y="+ yy +" AND m="+ mm +" AND d="+ dd +" AND ip_dst='"+ anchor +"' GROUP BY dns_qry_name, ip_dst" +
                   " LIMIT 1000\" > " + ir_f) 
        elif anchor_type == 'q':
            hivestr = ("\"set mapred.max.split.size=1073741824;set hive.exec.reducers.max=10;set hive.cli.print.header=true;" +
                   " SELECT COUNT(ip_dst) as total, dns_qry_name, ip_dst, 0 as sev FROM duxbury.dns " + 
                   " WHERE y="+ yy +" AND m="+ mm +" AND d="+ dd +" AND dns_qry_name='"+ anchor +"'" +
                   " GROUP BY ip_dst, dns_qry_name LIMIT 1000\" > " + ir_f)  

        !hive -S -e $hivestr           
        height=25
#       total, dns_qry_name, ip_dst, sev
        with open(ir_f, 'r') as f:
            reader = csv.reader(f, delimiter='\t') 
            if reader!= '':
                reader.next()
                for row in reader:  
                    table += "<tr><td>" +  row[2] + "</td><td>" + row[1] + "</td><td>" + str(row[0]) + "</td></tr>"  
                    height += 20
            table += "</table>"            
                    
        result_html.value=table
        result_box.height=max(200,height)
        display_threat_box(anchor)
        display(result_box)
    
    search_btn.on_click(search_ip)

        
def display_threat_box(ip):    
    tc_div_label = widgets.HTML(value='<h4>Threat summary for ' + anchor +'</h4>')
    
    tc_txt_title = widgets.Text(value='',width=300, placeholder='Threat Title')
    tc_txa_summary = widgets.Textarea(value='', width=300, height=100)
    tc_btn_save = widgets.Button(description='Save')
    threat_container.children = [tc_txa_summary,tc_btn_save]
    tc_box_main.children = [tc_div_label, tc_txt_title, tc_box_separator, threat_container]  

    display(tc_box_main)     
    
    def save_threat_summary(b):
        global anchor
        anchor_ip =''
        anchor_dns ='' 
        if anchor != '':             
            if anchor_type == 'i':
                anchor_ip = anchor
            elif anchor_type == 'q':
                anchor_dns = anchor
                
            global threat_f
            if not os.path.exists(threat_f):  
                with open(threat_f, 'w') as comment:
                    comment.write('ip_dst|dns_qry_name|title|summary\n')
            
            with open(threat_f, 'a') as comment:
                comment.write(anchor_ip + '|' + anchor_dns + '|' + tc_txt_title.value + '|' +
                                  tc_txa_summary.value.replace('\n', '\\n') + '\n') 
            
            removeWidget(2)
            removeWidget(1)  
            generate_dendro(anchor)
            display(Javascript("$(\"option[data-value='" + anchor  +"']\").remove();"))               
            response = "Successfully saved"
        else:
            response = "No data selected"
            
        save_html = widgets.HTML(value=response, width=500)
        save_box  = widgets.Box(width=500, height=150)
        save_box.children = [save_html]  
        susp_select.selected_label = susp_select.options[0][0]
        display(save_box)      
        
        
    tc_btn_save.on_click(save_threat_summary)
    
    
    
def generate_dendro(anchor):  
    dendro_fpath = spath + 'threat-dendro-' + anchor + ".csv" 
    #       total, dns_qry_name, ip_dst, sev
    headers=["dns_qry_name",  "ip_dst", "sev","total"]
    global ir_f
    global anchor_type 
    with open(ir_f, 'r') as f:
        reader = csv.reader(f, delimiter='\t') 
        if reader!= '':
            reader.next()            
            with open(dendro_fpath, 'w') as dendro_f:
                writer = csv.DictWriter(dendro_f, fieldnames=headers)
                writer.writeheader()
                if anchor_type == 'i':
                    for row in reader:
                        for key, sev in ip_sev.iteritems():
                            if key == row[2]:
                                writer.writerow({'dns_qry_name': row[0],'dns_qry_name': row[1], 'ip_dst': row[2], 'sev': sev}) 
                                break

                elif anchor_type =='q':
                    for row in reader:
                        for key, sev in dns_sev.iteritems():
                            if key == row[1]:
                                writer.writerow({'dns_qry_name': row[0],'dns_qry_name': row[1], 'ip_dst': row[2], 'sev': sev}) 
                                break

    print 'Incident progression successfully created'
    

def removeWidget(index):
    js_command = "var widget_list = $('.widget-area').children('.widget-subarea').children('.widget-box'); \n"
    js_command = js_command + "if(widget_list.length > {0} ) \n ".format(index)
    js_command = js_command + "widget_list[{0}].remove(); \n ".format(index)
    
    display(Javascript(js_command))  

In [26]:
start_investigation()

ls: cannot access ./usr/bin/pyspark/lib/spark-assembly-*.jar: No such file or directory
